In [ ]:
from typing import Optional, Iterable, Union, Callable
import pandas as pd
import numpy as np
from snowflake.snowpark import Session
session = Session.builder.getOrCreate()

In [ ]:
# ------------------ Minimal Obfuscation Framework ------------------
import hashlib, random
from typing import Optional

# ---------- Masking Rules ----------
def keep(value: Optional[str]) -> Optional[str]:
    return value

def nullify(value: Optional[str]) -> None:
    return None

def mask_fixed(value: Optional[str], mask_char: str = "X") -> Optional[str]:
    if value is None:
        return None
    s = str(value)
    return mask_char * len(s)

def mask_lastn(value: Optional[str], n: int = 4, mask_char: str = "X") -> Optional[str]:
    if value is None:
        return None
    s = str(value)
    if len(s) <= n:
        # return s
        return mask_char * len(s)   
    return (mask_char * (len(s) - n)) + s[-n:]

def mask_firstn(value: Optional[str], n: int = 4, mask_char: str = "X") -> Optional[str]:
    if value is None:
        return None
    s = str(value)
    if len(s) <= n:
        # return s
        return mask_char * len(s)   
    return s[:n] + (mask_char * (len(s) - n))

# ---------- Scrambling Helpers ----------
def _seed(s: str) -> int:
    return int(hashlib.sha256(s.encode("utf-8")).hexdigest(), 16)

def _internal_shuffle(s: str) -> str:
    if s is None or len(s) <= 2:
        return s
    first, last = s[0], s[-1]
    mid = list(s[1:-1])
    random.Random(_seed(s)).shuffle(mid)
    return first + "".join(mid) + last

def _full_shuffle(s: str) -> str:
    if s is None or len(s) <= 1:
        return s
    chars = list(s)
    random.Random(_seed(s)).shuffle(chars)
    return "".join(chars)

def _reverse(s: str) -> str:
    return None if s is None else s[::-1]

def obfuscate(text: str, variant: str = "INTERNAL") -> str:
    """
    variant: "INTERNAL" | "FULL" | "REVERSE"
    """
    if text is None:
        return None
    v = variant.upper()
    if v == "FULL":
        return _full_shuffle(text)
    if v == "REVERSE":
        return _reverse(text)
    # default INTERNAL (keep first/last, shuffle middle)
    return _internal_shuffle(text)

# ------------------ End of Framework ------------------

In [ ]:
# ------------------ Demo with Pandas DataFrame ------------------
values = ["pavan", "HOUSEHOLD", "1234567890", "Alice", "sam", "cutter"]
df = pd.DataFrame(values, columns=["Original"])

# Apply each rule/variant
df["KEEP"]        = df["Original"].apply(keep)
df["NULLIFY"]     = df["Original"].apply(nullify)
df["MASK_FIXED"]  = df["Original"].apply(lambda v: mask_fixed(v, "*"))
df["MASK_LASTN"]  = df["Original"].apply(lambda v: mask_lastn(v, 4, "X"))
df["MASK_FIRSTN"] = df["Original"].apply(lambda v: mask_firstn(v, 3, "#"))
df["OBF_INTERNAL"] = df["Original"].apply(lambda v: obfuscate(v, "INTERNAL"))
df["OBF_FULL"]     = df["Original"].apply(lambda v: obfuscate(v, "FULL"))
df["OBF_REVERSE"]  = df["Original"].apply(lambda v: obfuscate(v, "REVERSE"))

df